<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/scare_quote_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')
#TODO
# fix -- change to scare quotes folder
%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/1_Data Prep
%ls


/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/1_Data Prep


BigQuery

In [ ]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [ ]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


Notes and patients load

In [ ]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents
                LIMIT 50000''')

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



Explode by sentence, regex quote match

In [ ]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)




In [ ]:
# Read in tokenized data frame
full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")



<ipython-input-2-eb19bb3fafe2>:2: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")


In [ ]:
small_dataframe = full_dataframe.head(100000)

def check_text(text):
    pattern = r'(?=.*\".*\")(?=.*\b(pt|patient|pateint)\b)'
    return bool(re.search(pattern, text, re.IGNORECASE))

# Test
print(check_text('This is a "test" string with patient'))  # True
print(check_text('This is a "test" string without any term'))  # False
print(check_text('This is a test string with pt but without quotes'))  # False


regex = r'(?=.*\".*\")(?=.*\b(pt|patient|pateint)\b)'
quotes = r'"(.*?)"'
quote_only = full_dataframe.loc[full_dataframe['Sentence'].str.contains(quotes)]
print(f"Sentences of only double-quote matches: {quote_only.shape[0]}")  # Print the size before filter



# Add & needs to include patient | pt , lowercase match
quoted_dataframe = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex)]
quoted_dataframe['quoted_text'] = full_dataframe['Sentence'].str.extract(quotes)
print(f"Sentences of double-quote and patient matches: {quoted_dataframe.shape[0]}")  # Print the size after filter

quoted_dataframe
quoted_dataframe["scare_quote"] = ""
quoted_dataframe["annotator_comments"] = ""
# 3842 quoted charts sentences total of 1,510,650 sentences
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("quoted_dataframe_annotate.csv")



True
False
False


<ipython-input-32-0c24a31b9f94>:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  quote_only = full_dataframe.loc[full_dataframe['Sentence'].str.contains(quotes)]


Sentences of only double-quote matches: 42999
Sentences of double-quote and patient matches: 6814


# Sample the annotation dataframe

In [14]:
quoted_dataframe2 = pd.read_csv("quoted_dataframe_annotate.csv")
np.random.seed(23)
# List of tuples (sample size, dataframe name)
sample_params = [(50, 'Drew_initial'), (5, 'cocode'), (100, 'reliability'), (400, 'annotation_Annie'), (500, 'annotation_Drew')]

# Dictionary to hold the dataframes
dfs = {}

for size, name in sample_params:
    # Random sample without replacement
    sample_df = quoted_dataframe2.sample(n=size, replace=False)

    # Add the sample to the dictionary
    dfs[name] = sample_df

    # Remove the sampled rows from the original dataframe
    quoted_dataframe2 = quoted_dataframe2.drop(sample_df.index)

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/2_Annotation

# Now you can access each dataframe using their names.
# For example, to access the 'Drew' dataframe:
drew_df = dfs['Drew_initial']
drew_df
drew_df.to_excel("Drew_initial_sample_scare_quotes.xlsx", index=False)

cocode_df = dfs['cocode']
cocode_df
cocode_df.to_excel("cocode_sample_scare_quotes.xlsx", index=False)

reliability_df = dfs['reliability']
reliability_df
reliability_df.to_excel("reliability_sample_scare_quotes.xlsx", index = False)

annotation_Annie_df = dfs['annotation_Annie']
annotation_Annie_df
annotation_Annie_df.to_excel("annotation_Annie_sample_scare_quotes.xlsx", index = False)

annotation_Drew_df = dfs['annotation_Drew']
annotation_Drew_df
annotation_Drew_df.to_excel("annotation_Drew_sample_scare_quotes.xlsx", index = False)

quoted_dataframe2

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/2_Annotation


,Unnamed: 0,Sentence ID,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,...,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Sentence,quoted_text,scare_quote,annotator_comments
0,1140,1140,1417557,11861,188478.0,2126-06-17,2126-06-17 01:34:00,2126-06-17 01:45:00,Nursing/other,Report,...,F,2105-05-05,NaN,NaN,NaN,0,Nursing Assessment Note 1900-0700 NEURO: Pt A&...,feeling hot,NaN,NaN
1,1334,1334,1417524,11861,102760.0,2125-10-06,2125-10-06 17:12:00,2125-10-06 17:31:00,Nursing/other,Report,...,F,2105-05-05,NaN,NaN,NaN,0,"MICU/SICU NPN HD #2 S: ""My throat really hurts...",My throat really hurts,NaN,NaN
2,3586,3586,1417519,11861,110532.0,2125-09-23,2125-09-23 04:55:00,2125-09-23 05:30:00,Nursing/other,Report,...,F,2105-05-05,NaN,NaN,NaN,0,"pt describes it as a ""burning"" pain.",burning,NaN,NaN
3,4906,4906,1417532,11861,123134.0,2126-02-28,2126-02-28 06:32:00,2126-02-28 06:42:00,Nursing/other,Report,...,F,2105-05-05,NaN,NaN,NaN,0,IV repletion fluid D5NS changed to D5NS with ...,I feel 100%.,NaN,NaN
4,5755,5755,22376,11861,177702.0,2126-09-19,NaN,NaN,Discharge summary,Report,...,F,2105-05-05,NaN,NaN,NaN,0,"Later in the day, she checked her urine ketone...",large,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5149,18274834,18274834,1547823,21773,128838.0,2146-05-19,2146-05-19 15:39:00,2146-05-19 15:59:00,Nursing/other,Report,...,M,2091-10-07,NaN,NaN,NaN,0,"Cough pillow given, but pt stated that ""pillow...",pillow hurts abdomen more.,NaN,NaN
5150,18281432,18281432,1322639,4655,120724.0,2202-08-31,2202-08-31 04:38:00,2202-08-31 05:15:00,Nursing/other,Report,...,F,2163-08-26,2204-02-09,2204-02-09,2204-02-09,1,"At times paranoid, pt questioned staff if they...",cut off an arm and a leg or both legs?,NaN,NaN
5151,18282938,18282938,1322635,4655,120724.0,2202-08-24,2202-08-24 05:05:00,2202-08-24 05:21:00,Nursing/other,Report,...,F,2163-08-26,2204-02-09,2204-02-09,2204-02-09,1,npn 1a-7a dx: hypotension 38F e/ ESRD d/t lith...,Hematology/Oncology,NaN,NaN
5154,18286745,18286745,1567014,23318,159496.0,2150-04-01,2150-04-01 04:20:00,2150-04-01 04:32:00,Nursing/other,Report,...,F,2080-11-30,NaN,NaN,NaN,0,"CCU NPN 1900-0700 S: "" I feel so much better ""...",I feel so much better,NaN,NaN


Explore the Data


In [ ]:
from google.colab import runtime
runtime.unassign()